## 퍼셉트론

```
     - 신경망을 이루는 가장 중요한 기본 단위는 퍼셉트론
     - 퍼셉트론 : 입력 값과 활성화 함수를 사용해 출력 값을 다음으로 넘기는 가장 작은 신경망 단위
     
     - 기울기 = 가중치 (w), y절편 = 바이어스 (b)
     - 가중합의 결과를 놓고 1또는 0을 출력해서 다음으로 보냄
     - 활성화 함수를 통해 1과 0을 판단해 출력함
     
```

###  Single neural network AND 연산

In [4]:
import numpy as np
import tensorflow as tf

In [20]:
x_data = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
y_data = np.array([[0],[1],[1],[0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None,2],name='x_input')
Y = tf.placeholder(tf.float32, [None,1],name='y_input')

#가중치
W = tf.Variable(tf.random_normal([2,1], name='weight'))
#바이어스
b = tf.Variable(tf.random_normal([1]),name='bias')

#활성화 함수
hypothesis = tf.sigmoid(tf.matmul(X,W)+b)

#cost/loss
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

#예측값 결과 계산 및 정밀도 계산
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))


#Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data,Y:y_data})
        
        if step % 100 == 0:
            print("step = ",step,"cost= ",sess.run(cost,feed_dict={X:x_data,Y:y_data}), "W= ", sess.run(W), "b= ", sess.run(b))
            
    h,c,a=sess.run([hypothesis,predicted,accuracy], feed_dict ={X:x_data, Y:y_data})
    print("\nHypothesis:",h,"\nCorrect(y): ",c,"\n Accuracy:",a)
            
    


step =  0 cost=  1.057761 W=  [[1.1790476]
 [2.7920907]] b=  [-0.8022166]
step =  100 cost=  0.7744708 W=  [[0.6045825]
 [1.5046046]] b=  [-1.1998127]
step =  200 cost=  0.73048544 W=  [[0.4744897 ]
 [0.95994884]] b=  [-0.85397816]
step =  300 cost=  0.7101486 W=  [[0.36056906]
 [0.6208953 ]] b=  [-0.5834466]
step =  400 cost=  0.70084876 W=  [[0.2638228 ]
 [0.40309313]] b=  [-0.3959346]
step =  500 cost=  0.69663334 W=  [[0.18855013]
 [0.26299262]] b=  [-0.2679167]
step =  600 cost=  0.6947268 W=  [[0.13271523]
 [0.17249273]] b=  [-0.1810456]
step =  700 cost=  0.69386387 W=  [[0.09244291]
 [0.11369473]] b=  [-0.12226492]
step =  800 cost=  0.69347274 W=  [[0.06391151]
 [0.07526501]] b=  [-0.08254471]
step =  900 cost=  0.69329524 W=  [[0.0439432 ]
 [0.05000852]] b=  [-0.05572098]
step =  1000 cost=  0.69321454 W=  [[0.03008886]
 [0.03332905]] b=  [-0.0376116]
step =  1100 cost=  0.6931778 W=  [[0.02053753]
 [0.02226847]] b=  [-0.02538706]
step =  1200 cost=  0.69316113 W=  [[0.013984

## 다중 퍼셉트론 [ XOR ]

```  
    1. 따라서 만약 두개의 직선(다중)을 만들기 위해서 두개의 뉴런이 필요[hidden layer]
    2. 이 둘을 교차시키는 연산이 필요함. 
    3. 여기서 활성함수로는 시그모이드를 사용함. 
    
    
    * 여기서 AND,OR,NAND를 (hidden layer로 구성되는) 만족하는 가중치와 바이어스의 조합은 무수히 많음.
    
    y = w1*1 _ w2*2 + b
    
 
```

In [12]:
################   NUMPY로 짜보기   ############


#가중치와 바이어스
w11 = np.array([-7.40,-7.40])
w12 = np.array([8.67,8.67])
w2 = np.array([7.41,7.41])
b1 = 11.28
b2 = -2.87
b3 = -11.29

#퍼센트론
def MLP(x ,w ,b):
    y = np.sum(w*x) + b
    if y <= 0:
        return 0
    else:
        return 1
    
    
#NAND 게이트
def NAND(x1, x2):
    return MLP(np.array([x1,x2]),w11,b1)

def OR(x1, x2):
    return MLP(np.array([x1, x2]), w12, b2)

#AND 게이트
def AND(x1, x2):
    return MLP(np.array([x1, x2]), w2,b3)

def XOR(x1, x2):
    return AND(NAND(x1, x2), OR(x1,x2))


#x1, x2 값을 번갈아 대입해 가며 최종값 출력
if __name__ == '__main__':
    for x in [(0,0),(1,0),(0,1),(1,1)]:
        y = XOR(x[0],x[1])
        print("입력값 : "+ str(x) + "출력값:" + str(y))



입력값 : (0, 0)출력값:0
입력값 : (1, 0)출력값:1
입력값 : (0, 1)출력값:1
입력값 : (1, 1)출력값:0


## 오차 역전파

```
    신경망 내부의 가중치는 오차 역전파 방법을 사용해 수정
    * 가중치  --> 경사 하강법 사용!
                  실제값(y)이 나오면 출력층에서 시작해 뒤에서 앞(hidden layer 쪽)으로 진행됨 
                  이과정에서 가중치에서 기울기를 뺏을 때 가중치 변화가 없을 때까지 나아감.
                  --> 오차 역전파
    
```

In [18]:
learning_rate = 0.1
tf.set_random_seed(0)
np.random.seed(0)

x_data = [[0,0],[0,1],[1,0],[1,1]]
y_data=[[0],[1],[1],[0]]

X = tf.placeholder(tf.float32,[None,2])
Y = tf.placeholder(tf.float32,[None,1])

W1 = tf.Variable(tf.random_normal([2,2], name ='weight1'))
b1 = tf.Variable(tf.random_normal([2]), name = 'bias1')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)

W2 = tf.Variable(tf.random_normal([2,1]),name='weight2')
b2 = tf.Variable(tf.random_normal([1]),name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1,W2) + b2)

cost =-tf.reduce_mean(Y*tf.log(hypothesis) + (1+Y) *tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train,feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W2))
            
    h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis:\n",h, "\nCorrect:\n",c ,"\nAccuracy:\n",a)


0 1.1549672 [[-0.4460009]
 [-1.5890768]]
100 1.1343694 [[-0.49363706]
 [-1.6886524 ]]
200 1.1330063 [[-0.47585425]
 [-1.6574812 ]]
300 1.1318729 [[-0.45962474]
 [-1.6305039 ]]
400 1.1309117 [[-0.4446806]
 [-1.6070787]]
500 1.1300824 [[-0.43080032]
 [-1.586696  ]]
600 1.1293547 [[-0.4178094]
 [-1.5689653]]
700 1.1287065 [[-0.40556934]
 [-1.5535817 ]]
800 1.1281204 [[-0.39396918]
 [-1.5403051 ]]
900 1.1275835 [[-0.3829196]
 [-1.5289443]]
1000 1.127085 [[-0.37234983]
 [-1.5193428 ]]
1100 1.1266166 [[-0.36220458]
 [-1.5113732 ]]
1200 1.1261712 [[-0.3524423]
 [-1.5049274]]
1300 1.1257434 [[-0.34303394]
 [-1.4999119 ]]
1400 1.1253285 [[-0.3339632]
 [-1.4962431]]
1500 1.124923 [[-0.3252257]
 [-1.4938422]]
1600 1.1245238 [[-0.31682986]
 [-1.4926308 ]]
1700 1.1241288 [[-0.3087968]
 [-1.4925282]]
1800 1.1237363 [[-0.3011608]
 [-1.4934486]]
1900 1.1233456 [[-0.29397053]
 [-1.495296  ]]
2000 1.1229562 [[-0.28728884]
 [-1.4979644 ]]
2100 1.1225684 [[-0.2811928]
 [-1.501335 ]]
2200 1.1221824 [[-0.27